In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
index = pd.read_csv("index.csv", index_col=[0, 1])

index.dropna(inplace=True)
index.drop(index[index["housing"] == 0].index, inplace=True)

In [ ]:
biking = pd.read_csv("biking/biking.csv", index_col=0)
housing = pd.read_csv("housing/housing.csv", index_col=0)
vegan = pd.read_csv("vegan/vegan.csv", index_col=[0, 1])
weather = pd.read_csv("weather/weather.csv", index_col=[0])

data = pd.DataFrame(index=index.index)
data["biking"] = biking.loc[index["biking"], "per_100k"].values
data["housing"] = housing.loc[index["housing"], "zhvi"].values
data["vegan"] = vegan["per_100k"]
data["winter"] = weather.loc[index["weather"].astype("str"), "winter"].values
data["summer"] = weather.loc[index["weather"].astype("str"), "summer"].values
data.head()

In [ ]:
print(data.to_string())
data.to_csv("data.csv")

In [ ]:
chicago = data.loc[("Chicago", "Illinois")]

weights = {
    "biking": 1,
    "housing": -1,
    "vegan": 1,
    "winter": 0.5,
    "summer": 0.5,
}

scores = data
scores = scores.apply(lambda x: (x/chicago).apply(np.log2), axis=1)
scores *= weights

In [ ]:
chicago = scores.loc[("Chicago", "Illinois")]

scores["total"] = scores[["biking", "housing",
                          "vegan", "winter", "summer"]].mean(axis=1)
scores.sort_values("total", ascending=False, inplace=True)

In [ ]:
scores = scores.replace([np.inf, -np.inf], np.nan).dropna()
print(scores.to_string())
scores.to_csv("scores.csv")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(scores["housing"], scores[["biking", "vegan", "winter", "summer"]].mean(axis=1), "o")
ax.set_xlabel("housing")
ax.set_ylabel("biking + vegan + winter")
ax.grid()

In [ ]:
scores.corr()